In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(".")
sys.path.append("..")
from pathlib import Path
from src.datamodule.av2_dataset import Av2Dataset
from src.datamodule.av2_extractor import Av2Extractor
from src.datamodule.av2_dataset import collate_fn
from av2.datasets.motion_forecasting import scenario_serialization
from av2.map.map_api import ArgoverseStaticMap
from src.utils.vis import  visualize_scenario
from src.model.multiagent.trainer_multiagent_mae import Trainer as Model
from torch.utils.data import DataLoader as TorchDataLoader

import torch
from typing import Dict
import matplotlib.pyplot as plt
import numpy as np
import tqdm

/home/jerome.zhou/anaconda3/envs/predict_py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data_root = Path("/data/jerome.zhou/prediction_dataset/av2/multiagent-baseline") # Path to the datase
ckpt = "../outputs/atpn_test_baseline_scene_q/2024-03-27/19-11-00/checkpoints/last.ckpt"
cached_split =  "val"
dataset = Av2Dataset(data_root=data_root, cached_split="val", extractor=Av2Extractor(radius=150))
model = Model.load_from_checkpoint(ckpt, pretrain_weights=None)
model = model.eval()

data root: /data/jerome.zhou/prediction_dataset/av2/multiagent-baseline/val, total number of files: 24988


In [4]:
data = dataset[16]
scene_id = data["scenario_id"]
data_root = Path("/data/jerome.zhou/prediction_dataset/av2/")
scene_file = data_root / cached_split / scene_id / ("scenario_" + scene_id +
                                                    ".parquet")
map_file = data_root / cached_split / scene_id / ("log_map_archive_" +
                                                  scene_id + ".json")
scenario = scenario_serialization.load_argoverse_scenario_parquet(scene_file)
static_map = ArgoverseStaticMap.from_json(map_file)
dataloader = TorchDataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    collate_fn=collate_fn,
)


In [38]:
for i, data in tqdm.tqdm(enumerate(dataloader)):
  # if i >= 2:
  #   break
  predictions = model(data)
  viz_batched_data(data, predictions)
  # print(f"Epoch: {i}")
  # Viz Scene use api
  # scene_id = data["scenario_id"][0]
  # scene_file = data_root / cached_split / scene_id / ("scenario_" + scene_id +
  #                                                   ".parquet")
  # map_file = data_root / cached_split / scene_id / ("log_map_archive_" +
  #                                                 scene_id + ".json")
  # scenario = scenario_serialization.load_argoverse_scenario_parquet(scene_file)
  # static_map = ArgoverseStaticMap.from_json(map_file)
  # visualize_scenario(scenario, static_map)

20it [00:44,  2.26s/it]/tmp/ipykernel_77681/2605766975.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(27, 9))
2126it [1:15:06,  1.67s/it]

In [36]:
def viz_batched_data(data, predictions):
    B,N,T,_= data['x'].shape
    _,M,L,_ =data["lane_positions"].shape
    # each batch is a single sceneario
    for bs in range(B):
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(27, 9))
        # Plot trajectory
        hist_traj = data['x_positions'][bs]
        y = data['y'][bs]
        padding_mask = data['x_padding_mask'][bs]
        lane_padding_mask = data["lane_padding_mask"][bs]
        for n in range(N):
            hist_traj_valid = hist_traj[n][:T][~padding_mask[n][:T]]
            ax1.plot(hist_traj_valid[:,0], hist_traj_valid[:,1], linestyle='-', color='g', alpha=1)
            ax2.plot(hist_traj_valid[:,0], hist_traj_valid[:,1], linestyle='-', color='g', alpha=1)
            ax3.plot(hist_traj_valid[:,0], hist_traj_valid[:,1], linestyle='-', color='g', alpha=1)
            y_traj_valid = y[n][~padding_mask[n][T:]] + hist_traj[n][49]
            ax1.plot(y_traj_valid[:,0], y_traj_valid[:,1], linestyle='-', color='r', alpha=1)
            ax2.plot(y_traj_valid[:,0], y_traj_valid[:,1], linestyle='-', color='r', alpha=1)
            ax3.plot(y_traj_valid[:,0], y_traj_valid[:,1], linestyle='-', color='r', alpha=1)
        # Plot road map
        road = data["lane_positions"][bs]
        for m in range(M):
            road_valid = road[m][~lane_padding_mask[m][:L]]
            ax1.plot(road_valid[:,0], road_valid[:,1], linestyle='-', color='grey', alpha=0.7)
            ax2.plot(road_valid[:,0], road_valid[:,1], linestyle='-', color='grey', alpha=0.7)
            ax3.plot(road_valid[:,0], road_valid[:,1], linestyle='-', color='grey', alpha=0.7)

        # Plot predictions result
        pred_traj = predictions['y_hat'][bs].detach()
        pred_score = predictions['pi'][bs].detach()
        pred_score = torch.softmax(pred_score.squeeze().double(), dim=-1)
        sorted_indices = torch.argsort(pred_score)
        ranks = torch.empty_like(sorted_indices)
        ranks[sorted_indices] = torch.arange(len(pred_score))
        colors = ['y', 'y', 'y', 'y', 'y', 'b']

        x_scored = data['x_scored'][bs]
        _,K,fut_steps,_=pred_traj.shape
        for n in range(N):
            for k in range(K):
                alpha = 1 if ranks[k] == 5 else 0.5
                ax2.plot(pred_traj[n,k,:,0] + hist_traj[n, 49, 0], pred_traj[n,k,:,1]+hist_traj[n, 49, 1], \
                         linestyle='-', color=(colors[ranks[k]]), alpha=alpha)
                ax2.plot((pred_traj[n,k,:,0] + hist_traj[n, 49, 0])[-1], \
                                     (pred_traj[n,k,:,1]+hist_traj[n, 49, 1])[-1], 'o', color=(colors[ranks[k]]), markersize=2.5, alpha=alpha)
                # ax2.text((pred_traj[n,k,:,0] + hist_traj[n, 49, 0])[-1], (pred_traj[n,k,:,1]+hist_traj[n, 49, 1])[-1], f"{pred_score[k]:.2f}", fontsize=4)
                if x_scored[n]:
                    ax3.plot(pred_traj[n,k,:,0] + hist_traj[n, 49, 0], pred_traj[n,k,:,1]+hist_traj[n, 49, 1], \
                         linestyle='-', color=(colors[ranks[k]]), alpha=alpha)
                    ax3.plot((pred_traj[n,k,:,0] + hist_traj[n, 49, 0])[-1], \
                                        (pred_traj[n,k,:,1]+hist_traj[n, 49, 1])[-1], 'o', color=(colors[ranks[k]]), markersize=2.5, alpha=alpha)
                    # ax3.text((pred_traj[n,k,:,0] + hist_traj[n, 49, 0])[-1], (pred_traj[n,k,:,1]+hist_traj[n, 49, 1])[-1], f"{pred_score[k]:.2f}", fontsize=4)

        
        ax1.set_title('Scene')
        ax2.set_title('Prediction')
        ax3.set_title('Prediction Scored')

        plt.savefig(f"../viz/multiagent/{data['scenario_id'][bs]}.png")